In [1]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [2]:
#load dataset yang akan digunakan untuk test
data = pd.read_csv('data\data.csv')

menyiapkan dataset untuk testing

In [3]:
data.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


menampilkan lima data pertama

In [9]:
text_test = data['text'][13]

text_test

"No place in my city has shelter space for us, and I won't put my baby on the literal street. (I've been there before. Fine for new, but I don't want to have to go through that with a baby). What cities have good shelter programs for homeless mothers and children? I'd love to be able to make some calls and get on a greyhound between the 7th and 16th (when I'll likely be able to get the money to go)."

kode diatas mengambil data bari ke 13 untuk dilakukan testing

In [10]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)
    
endpoint = "http://localhost:8080/v1/models/humanstress-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Stress" if prediction_value > 0.6 else "Tidak Stress"
else:
    result = "Error: No predictions found in response."

print(result)

Stress


prepare_json() untuk mempersiapkan data input dalam json string yang akan digunakan untuk prediksi

Melakukan prediksi dari text_test yang dideploy pada api endpoint, apabila prediksi lebih besar dari 0.6 maka akan menghasilkan prediksi dari text "Stress", apabila dibawah 0.6 maka tidak stress